In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [2]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/informacion_artista.csv")

display(df.head())

,artista_id,pais_de_origen,area_de_origen,fecha_nacimiento,fecha_inicio_actividad,fecha_fin_actividad
0,747,United States,New York,NaN,NaN,Actualidad
1,744,United States,Wasilla,NaN,2004.0,Actualidad
2,159,Sweden,Stockholm,NaN,NaN,Actualidad
3,426,Spain,Alcalá de Henares,1985-07-30,NaN,Actualidad
4,631,United States,San Francisco,NaN,2015.0,Actualidad


In [3]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 819 filas y 6 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 4 columnas con datos nulos, y son:
['pais_de_origen', 'area_de_origen', 'fecha_nacimiento', 'fecha_inicio_actividad']
y sin nulos hay 2 columnas y son:
['artista_id', 'fecha_fin_actividad']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
artista_id,0.0%,100.0%,int64,819
pais_de_origen,9.65%,90.35%,object,35
area_de_origen,25.76%,74.24%,object,369
fecha_nacimiento,58.73%,41.27%,object,329
fecha_inicio_actividad,66.54%,33.46%,float64,53


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
pais_de_origen,740,35,United States,281
area_de_origen,608,369,Madrid,27
fecha_nacimiento,338,329,1996-01-01,3
fecha_fin_actividad,819,29,Actualidad,790


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
artista_id,819.0,452.223443,259.925827,1.0,226.5,457.0,674.5,899.0
fecha_inicio_actividad,274.0,2001.109489,11.725292,1958.0,1995.0,2002.5,2010.0,2021.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
artista_id,0.0%,100.0%,int64,819
pais_de_origen,9.65%,90.35%,object,35
area_de_origen,25.76%,74.24%,object,369
fecha_nacimiento,58.73%,41.27%,object,329
fecha_inicio_actividad,66.54%,33.46%,float64,53
fecha_fin_actividad,0.0%,100.0%,object,29


# Valores unicos de las variables categoricas ✨

In [4]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['pais_de_origen', 'area_de_origen', 'fecha_nacimiento', 'fecha_fin_actividad']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'PAIS_DE_ORIGEN' -----------

Sus valores únicos son: ['United States' 'Sweden' 'Spain' 'Mexico' 'Germany' 'United Kingdom'
 'Colombia' nan 'Ireland' 'Argentina' 'France' 'Japan' 'Australia'
 'Netherlands' 'Norway' 'Poland' 'Venezuela' 'Denmark' 'Iceland' 'Italy'
 'Chile' 'Cuba' 'Austria' 'Canada' 'Belgium' 'Greece' 'Costa Rica'
 'New Zealand' 'Brazil' 'Philippines' 'Panama' 'Dominican Republic'
 'Algeria' 'South Korea' 'Jamaica' 'Guatemala']

Las frecuencias de los valores únicos de las categorías son: pais_de_origen
United States         281
Spain                 185
United Kingdom         80
Argentina              24
Colombia               22
Canada                 17
Japan                  15
Germany                14
France                 10
Australia               9
Italy                   8
Venezuela               8
Sweden                  7
Mexico       

# DUPLICADOS

In [5]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas completas duplicadas de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas duplicadas completas.
    """
    # Identificar duplicados basados en todas las columnas
    duplicate_rows = df[df.duplicated(keep=False)]
    return duplicate_rows

get_duplicate_rows(df)

,artista_id,pais_de_origen,area_de_origen,fecha_nacimiento,fecha_inicio_actividad,fecha_fin_actividad


In [6]:
# index=False es útil si solo quieres los datos y no necesitas la columna de índice.
#df.to_csv("archivos/informacion_artista_MOD.csv", index=False)

In [7]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/info_canciones.csv")

display(df.head())

,artista_id,nombre_cancion_album,tipo_album_cancion,año_lanzamiento,género
0,1,Seventeen,album,2022,Electrónica
1,2,Crust,album,2021,Electrónica
2,3,Abu Simbel,single,2021,Electrónica
3,4,Apricots,album,2021,Electrónica
4,5,KILL DEM,single,2022,Electrónica


In [8]:
exploracion(df)

El DataFrame tiene 4000 filas y 5 columnas.
Tiene 49 datos duplicados, lo que supone un porcentaje de 0.01% de los datos.
Hay 0 columnas con datos nulos, y son:
[]
y sin nulos hay 5 columnas y son:
['artista_id', 'nombre_cancion_album', 'tipo_album_cancion', 'año_lanzamiento', 'género']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
artista_id,0.0%,100.0%,int64,899
nombre_cancion_album,0.0%,100.0%,object,3563
tipo_album_cancion,0.0%,100.0%,object,3
año_lanzamiento,0.0%,100.0%,int64,2
género,0.0%,100.0%,object,4


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
nombre_cancion_album,4000,3563,Perra,4
tipo_album_cancion,4000,3,album,2508
género,4000,4,Electrónica,1000


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
artista_id,4000.0,323.220,239.358121,1.0,143.0,291.0,542.25,899.0
año_lanzamiento,4000.0,2021.544,0.498123,2021.0,2021.0,2022.0,2022.00,2022.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
artista_id,0.0%,100.0%,int64,899
nombre_cancion_album,0.0%,100.0%,object,3563
tipo_album_cancion,0.0%,100.0%,object,3
año_lanzamiento,0.0%,100.0%,int64,2
género,0.0%,100.0%,object,4


In [9]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['nombre_cancion_album', 'tipo_album_cancion', 'género']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'NOMBRE_CANCION_ALBUM' -----------

Sus valores únicos son: ['Seventeen' 'Crust' 'Abu Simbel' ... 'Maybe You Saved Me' 'homeswitcher'
 'Summertime']

Las frecuencias de los valores únicos de las categorías son: nombre_cancion_album
Perra            4
Run              3
Nena             3
Nena Buena       3
Nosotros         3
                ..
LINDOR           1
Hp Freestyle     1
Nos Matamos      1
Más De La Una    1
Sicaria          1
Name: count, Length: 3563, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'TIPO_ALBUM_CANCION' -----------

Sus valores únicos son: ['album' 'single' 'compilation']

Las frecuencias de los valores únicos de las categorías son: tipo_album_cancion
album          2508
single         1446
compilation      46
Name: count, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'GÉNERO' -----------

Sus valores únicos son: ['Electrónica' 'Latin